# Aplicació Mamografia

#### Setup environment

Instal·lem i importem les llibreries necessàries

In [5]:
pip install -q typeguard~=2.12.1  

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Install necessary packages for MONAI Core
!python -c "import monai" || pip install -q "monai[pillow, tqdm]"
!python -c "import ignite" || pip install -q "monai[ignite]"
!python -c "import gdown" || pip install -q "monai[gdown]"

# Install MONAI Deploy App SDK package
!python -c "import monai.deploy" || pip install -q "monai-deploy-app-sdk"

#### Setup imports

In [1]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import tempfile
import glob
import PIL.Image
import torch
import numpy as np
from pathlib import Path
import pandas as pd

from ignite.engine import Events
from tqdm import tqdm as notebook_tqdm

from monai.apps import download_and_extract
from monai.config import print_config
from monai.networks.nets import DenseNet121
from monai.engines import SupervisedTrainer
from monai.transforms import (
    AddChannel,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    Resize,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism

set_determinism(seed=0)

print_config()

/home/ursula/anaconda3/envs/mednist/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MONAI version: 0.6.0
Numpy version: 1.21.6
Pytorch version: 1.9.0+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 0ad9e73639e30f4f1af5a1f4a45da9cb09930179

Optional dependencies:
Pytorch Ignite version: 0.4.5
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.5.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: 4.7.1
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.3
pandas version: 1.3.5
einops version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
print(torch.cuda.is_available())

True


#### Load dataset

Definim la direcció a la carpeta de les imatges 

In [3]:
root_dir = "rsna22_bal/images_png"
print(root_dir)

#data_dir = os.path.join(root_dir, "train_image")
#if not os.path.exists(data_dir):
 #   os.makedirs(data_dir)

rsna22_bal/images_png


Carreguem les imatges i les etiquetes de cada classe. També veurem quantes imatges hi ha de cada classe i les mides. 

In [4]:
subdirs = sorted(glob.glob(f"{root_dir}/*/"))

class_names = [os.path.basename(sd[:-1]) for sd in subdirs]
image_files = [glob.glob(f"{sb}/*") for sb in subdirs]

image_files_list = sum(image_files, [])
image_class = sum(([i] * len(f) for i, f in enumerate(image_files)), [])
image_width, image_height = PIL.Image.open(image_files_list[0]).size

print(f"Label names: {class_names}")
print(f"Label counts: {list(map(len, image_files))}")
print(f"Total image count: {len(image_class)}")
print(f"Image dimensions: {image_width} x {image_height}")

Label names: ['Cancer', 'Normal']
Label counts: [709, 2058]
Total image count: 2767
Image dimensions: 918 x 1803


#### Setup and train

Definim una sèrie de transformacions per aplicar-les a les imatges, com retacions, redimensionar, fer zoom, ...

In [5]:
train_transforms = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        ScaleIntensity(),
        Resize((768, 768)),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
        EnsureType(),
    ]
)

Definim una classe Dataset que encapsularà un conjunt de dades personalitzades per entrenar. Cada element del conjunt de dades és una imatge transformada amb la seva etiqueta corresponent. Després creem un DataLoader per carregar les dades d’entrenament, hi assignem el batch size, que barregi aleatòriament les dades i que utilitzi ‘treballadors’ per carregar les dades en paral·lel.  

In [6]:
class MAMODataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]


# just one dataset and loader, we won't bother with validation or testing 
train_ds = MAMODataset(image_files_list, image_class, train_transforms)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=10)

Lliberem memòria que no està en ús

In [7]:
torch.cuda.empty_cache()

Configurem que volem que el codi s’executi a la GPU, definim quina arquitectura i loss function valem utilitzar, un optimitzador i les epochs.

In [8]:
device = torch.device("cuda:0")
net = DenseNet121(spatial_dims=2, in_channels=1, out_channels=len(class_names)).to(device)
loss_function = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), 1e-5)
max_epochs = 3

Definim un entrenador, realitzem l’entrenament i mostrem la loss (pèrdua) i la precisió a cada epoch (època). 

In [9]:
from sklearn.metrics import accuracy_score

def _prepare_batch(batch, device, non_blocking):
    return tuple(b.to(device) for b in batch)


trainer = SupervisedTrainer(device, max_epochs, train_loader, net, opt, loss_function, prepare_batch=_prepare_batch)

true_labels = []
predicted_labels = []


@trainer.on(Events.EPOCH_COMPLETED)
def _print_loss(engine):
    for batch in train_loader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        #Obtenim prediccions
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        
        true_labels.extend(labels.tolist())
        predicted_labels.extend(predicted.tolist())
        
    print(f"Epoch {engine.state.epoch}/{engine.state.max_epochs} Loss: {engine.state.output[0]['loss']}")
    #Calcular i mostrar l'accuracy de l'entrenament
    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f"Accuracy: {accuracy}")
    


trainer.run()

/home/ursula/anaconda3/envs/mednist/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 1/3 Loss: 0.24235236644744873
Accuracy: 0.7437658113480303
Epoch 2/3 Loss: 0.3186933398246765
Accuracy: 0.7437658113480303
Epoch 3/3 Loss: 0.2370697259902954
Accuracy: 0.7437658113480303


Emmagatzemem el model en un arxiu zip

In [10]:
torch.jit.script(net).save("mamoclassifier.zip")

## Implementing and Packaging Application with MONAI Deploy App SDK

#### Setup imports

Importem les llibreries necessàries i definim les etiquetes 

In [11]:
import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MAMO_CLASSES = ["Cancer", "Normal"]

#### Creating Operator classes

##### LoadPILOperator

Definim un operador que carregarà una imatge, la passarà a escala de grisos i la guardarà com un objecte d’imatge. L’operador s’utilitzarà per manipular i processar imatges. 

In [12]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        op_output.set(output_image)

##### MAMOClassifierOperator

Definim un altre operador que classifica una imatge utilitzant un model prèviament carregat i retorna el nom de la classe predita. L’operador agafa una imatge d’entrada, aplica les transformacions, realitza la inferència del model i guarda el resultat en un arxiu JSON. 

In [13]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MAMOClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MAMO_CLASSES[output_classes[0]]  # get the class name
        print(result)

        # Get output (folder) path and create the folder if not exists
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)

#### Creating Application class

Definim una aplicació que carrega una imatge utilitzant l’operador ‘LoadPILOperator’ i després la classifica utilitzant l’operador ‘MAMOClassifierOperator’. L’aplicació defineix els recursos necessaris i les dependències de l’entorn. 

In [14]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
@md.env(pip_packages=["pydicom"])
class App(Application):
    """Application class for the ECO classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MAMOClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)

### Executing app locally

Definim una imatge de prova

In [15]:
test_input_path = image_files[0][0]
print(f"Test input file path: {test_input_path}")

Test input file path: rsna22_bal/images_png/Cancer/44709_650076091_L.png


Executem l'aplicació per veure si funciona

In [16]:
app = App()

In [17]:
app.run(input=test_input_path, output="output", model="mamoclassifier.zip")

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 3942558, Operator ID: 36836862-6180-4955-9014-d21acaae0a85)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MAMOClassifierOperator
Executing operator MAMOClassifierOperator (Process ID: 3942558, Operator ID: 6b81fb78-b3ce-40ce-b53c-294790f8e974)
Normal
Done performing execution of operator MAMOClassifierOperator



Ens retorna un arxiu JSON amb l'etiqueta predita 

In [18]:
!cat output/output.json

"Normal"

Una vegada hem provat que l’aplicació funciona, ho podem escriure tota l’aplicació com a fitxer, concatenant el codi anterior.

In [19]:
%%writefile mamo_classifier_monaideploy.py

# Copyright 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MAMO_CLASSES = ["Cancer", "Normal"]


@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        op_output.set(output_image)


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class MAMOClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = op_input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MAMO_CLASSES[output_classes[0]]  # get the class name
        print(result)

        # Get output (folder) path and create the folder if not exists
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)


@md.resource(cpu=1, gpu=1, memory="1Gi")
@md.env(pip_packages=["monai","pydicom","highdicom","typeguard~=2.12.1","numpy==1.21.6"])
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MAMOClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)


if __name__ == "__main__":
    App(do_run=True)

Writing mamo_classifier_monaideploy.py


Ara, executarem l’aplicació des de la línia de comandaments. 

In [20]:
!python mamo_classifier_monaideploy.py -i {test_input_path} -o output -m mamoclassifier.zip

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 3951220, Operator ID: 25ff0937-9c00-49ad-8a54-d033e0f0a930)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MAMOClassifierOperator
Executing operator MAMOClassifierOperator (Process ID: 3951220, Operator ID: 2af162e3-080a-4d11-86b9-0a23e2d56f5a)
/home/ursula/anaconda3/envs/mednist/lib/python3.7/site-packages/torch/nn/modules/module.py:1051: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return forward_call(*input, **kwargs)
Normal
Done performing execution of operator MAMOClassifierOperator



In [21]:
!monai-deploy exec mamo_classifier_monaideploy.py -i {test_input_path} -o output -m mamoclassifier.zip

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 3951328, Operator ID: 886a8a9d-7df8-4a29-97b6-e300919f7a13)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MAMOClassifierOperator
Executing operator MAMOClassifierOperator (Process ID: 3951328, Operator ID: f41ce799-a85a-4d46-a107-b97efbe4a756)
/home/ursula/anaconda3/envs/mednist/lib/python3.7/site-packages/torch/nn/modules/module.py:1051: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return forward_call(*input, **kwargs)
Normal
Done performing execution of operator MAMOClassifierOperator



In [22]:
!cat output/output.json

"Normal"

### Packaging app

Empaquetarem l’aplicació en un contenidor Docker, on hi haurà l’arxiu zip del model i l’arxiu Python de l’aplicació

In [23]:
!monai-deploy package mamo_classifier_monaideploy.py --tag mamo_app:latest --model mamoclassifier.zip

Building MONAI Application Package... -[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (3/6)                                                         
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 1.11kB                                        0.0s
 => [internal] load build definition from dockerfile                       0.1s
 => => transferring dockerfile: 2.48kB                                     0.0s
 => [internal] load metadata for nvcr.io/nvidia/pytorch:21.07-py3          0.0s
[?25\[+] Building 0.2s (4/19)                                                        
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 1.11kB                                        0.0s
 => [internal] load build definition from dockerfile                       0.1s
 => => transferring dockerfile: 2.48kB                                  

Comprovem que s'ha creat la imatge Docker

In [24]:
!docker image ls | grep mamo_app

mamo_app                                   latest                           f80a3e55efd1   3 seconds ago    15.7GB


### Executing packaged app locally

Podem executar l'aplicació localment amb la seguent comanda 

In [25]:
# Copy a test input file to 'input' folder
!mkdir -p input && rm -rf input/*
!cp {test_input_path} input/

# Launch the app
!monai-deploy run mamo_app:latest input output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "mamo_app:latest" is available...

Checking for MAP "mamo_app:latest" locally
"mamo_app:latest" found.

Reading MONAI App Package manifest...
Preparing to copy...Copying from container - 0BSuccessfully copied 2.05kB to /tmp/tmpnxwoyjy_/app.json
Preparing to copy...Copying from container - 0BSuccessfully copied 2.05kB to /tmp/tmpnxwoyjy_/pkg.json
--> Verifying if "nvidia-docker" is installed...

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 1, Operator ID: f5fe3abf-58ff-4b76-ab34-8964ce379dfd)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MAMOClassifierOperator
Executing operator MAMOClassifierOperator (Process ID: 1, Operator ID: d8f91b03-dfb4-4021-a7a7-dac13a3c1c9a)
/root/.local/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.A

In [26]:
!cat output/output.json

"Normal"